In [ ]:
import fasttext
import json
import tqdm

In [ ]:
%cd ..
!mkdir pretrain

In [ ]:
!./build/tgnews json data > pretrain/data_raw.json

In [ ]:
TRAIN_FILE = 'markup/ru_cat_markup.txt'
TEST_FILE = 'markup/ru_cat_test_markup.txt'

RAW_DATA_FILE = 'pretrain/data_raw.json'
DATASET_FILE = 'pretrain/dataset.tsv'

In [ ]:
with open(RAW_DATA_FILE, 'r') as f:
    articles = json.load(f)
    
with open(DATASET_FILE, 'w') as out:
    for article in tqdm.tqdm(articles):
        title = article['title']
        text = article['text'].replace('\n', ' ').strip(' .!?')
        out.write('{}. {}.\n'.format(title, text))

params = {
    'model': 'skipgram',
    'dim': 64,
    'minn': 0,
    'maxn': 0,
    'epoch': 1,
    'verbose': 2,
    'thread': 4
}

WORD_EMBEDDINGS_FILE = 'model_filename.bin'

model = fasttext.train_unsupervised(DATASET_FILE, **params)
model.save_model(WORD_EMBEDDINGS_FILE)

In [ ]:
supervised_params = {
    'lr': 1.0,
    'epoch': 100,
    'minCount': 5,
    'ws': 3,
    'dim': 64
}

In [ ]:
supervised_model = fasttext.train_supervised(TRAIN_FILE, **supervised_params)
print(supervised_model.test(TEST_FILE))
print(json.dumps(supervised_model.test_label(TEST_FILE), indent=4))

In [ ]:
supervised_params_2 = supervised_params
supervised_params_2.update({
    'pretrainedVectors': 'pretrain/models/model_64_f15zQ.vec'
})

In [ ]:
supervised_model_2 = fasttext.train_supervised(TRAIN_FILE, **supervised_params_2)
print(supervised_model_2.test(TEST_FILE))
print(json.dumps(supervised_model_2.test_label(TEST_FILE), indent=4))

In [ ]:
m = fasttext.load_model('pretrain/models/model_64_f32zQ.bin')
print('words', len(m.words))